## Initial Setup
In this section we'll import the requisite libraries and instantiate a number of objects and variables to configure our training job

In [2]:
import sagemaker                                        # SageMaker Python SDK
from sagemaker.pytorch import PyTorch                   # PyTorch Estimator for running pytorch training jobs
from sagemaker.debugger import TensorBoardOutputConfig  # Debugger TensorBoard config to log training metrics to TensorBoard
import boto3                                            # AWS SDK for Python
import os
import tarfile
import pandas as pd
from pathlib import Path

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()   # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts
model_bucket = sess.default_bucket()  # bucket to house artifacts
s3_key_prefix = "flan-t5-finetune-for-dialogue"  # folder within bucket where code artifact will go

region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

s3_client = boto3.client("s3")  # client to intreract with S3 API
sm_client = boto3.client("sagemaker")  # client to intreract with SageMaker
smr_client = boto3.client("sagemaker-runtime")  # client to intreract with SageMaker Endpoints

In [ ]:
!wget https://raw.githubusercontent.com/cylnlp/dialogsum/main/DialogSum_Data/dialogsum.train.jsonl -O data/dialogsum.train.jsonl
!wget https://raw.githubusercontent.com/cylnlp/dialogsum/main/DialogSum_Data/dialogsum.test.jsonl -O data/dialogsum.test.jsonl

Let's take a look at a few examples

The `accelerate launch` command has two key parts, the `config.yml` file and the `train.py` script. The `config.yml` file is used to configure the distributed training job. The `train.py` script is the training script that will be launched by the launcher. In this example, we'll use the [ds_zero3.yml](src/train/ds_zero3.yaml) configuration file. The config file enables [DeepSpeed ZeRo Stage3](#https://www.deepspeed.ai/tutorials/zero/) and a number of other optimizations to enable training of large scale models. This file was generated by running `accelerate config --config_file ds_zero3.yml` and then following the on-screen prompts. 
The [train.py](src/train/train.py) makes use of a number of key libraries to enable training of large models with minimal code changes:
- 🤗 [Accelerate](https://huggingface.co/docs/accelerate/index) - Configures the distributed training environment and adapts training objects (data loaders, models, optimizers) to the distributed environment
- 🤗 [Transformers](https://huggingface.co/docs/transformers/index) - Provides a number of pre-trained models and utilities for training and evaluating models
- 🤗 [PEFT](https://github.com/huggingface/peft) - Provides a number of methods for Parameter Efficient Finetuning(PEFT) of large language models. The [LoRA](https://arxiv.org/pdf/2106.09685.pdf) method will be used to finetune the model
- [DeepSpeed](https://github.com/microsoft/DeepSpeed) - Provides a number of optimizations to enable training of large models. In this example, we'll use DeepSpeed ZeRO Stage3 to enable training of models with over 1B parameters

# 8 gpu

In [8]:
# configure the tesnorboard output directly to S3
tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path=f"s3://{bucket}/{s3_key_prefix}/tensorboard"
)

image = '763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:2.0.0-gpu-py310-cu118-ubuntu20.04-sagemaker'
estimator1 = PyTorch(
    source_dir = "src/train",
    entry_point="acc_launcher.py",
    role=role,
    instance_count=1, 
    instance_type="ml.p4d.24xlarge", 
    framework_version="2.0.0",
    py_version="py310",
    disable_profiler=True,
    tensorboard_output_config=tensorboard_output_config,
    hyperparameters = {
    "training_script": "train_qlora.py",
    "config_file": "qlora.yaml",
    "seed": 100,
    "model_name_or_path": "NousResearch/Llama-2-13b-hf",
    "dataset_name": "smangrul/ultrachat-10k-chatml",
    "chat_template_format": "chatml",
    "add_special_tokens": False,
    "append_concat_token": False,
    "splits": "train,test",
    "max_seq_len": 2048,
    "num_train_epochs": 1,
    "logging_strategy": "steps",
    "evaluation_strategy": "epoch",
    "bf16": True,
    "packing": True,
    "learning_rate": 1e-4,
    "lr_scheduler_type": "cosine",
    "weight_decay": 1e-4,
    "warmup_ratio": 0.0,
    "max_grad_norm": 1.0,
    "output_dir": "llama-sft-qlora-dsz3",
    "per_device_train_batch_size": 2,
    "per_device_eval_batch_size": 2,
    "gradient_accumulation_steps": 2,
    "gradient_checkpointing": True,
    "use_reentrant": True,
    "dataset_text_field": "content",
    "use_flash_attn": False,
    "use_peft_lora": True,
    "lora_r": 8,
    "lora_alpha": 16,
    "lora_dropout": 0.1,
    "lora_target_modules": "all-linear",
    "use_4bit_quantization": True,
    "use_nested_quant": True,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage_dtype": "bfloat16",
    "report_to":"none",

},

    keep_alive_period_in_seconds=3600
)

In [ ]:
estimator1.fit(wait=True)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2024-04-14-05-52-50-967


2024-04-14 05:52:51 Starting - Starting the training job...
2024-04-14 05:53:20 Pending - Preparing the instances for training.........

# 8 gpu

### Optional Section
#### Monitor the training with TensorBoard
**Note: You have to wait a few minutes for the job to launch before seeing any logs**

We can use [TensorBoard](https://www.tensorflow.org/tensorboard), a visualization toolkit for analyzing deep learning models to monitor the progress of the training. Instructions for using TensorBoard with SageMaker Studio can be found [here](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-tensorboard.html). Instructions for accessing TensorBoard in SageMaker Studio are provided below:
1. Open a new terminal in SageMaker Studio by navigating to <em>File->New->Terminal <br> ![](./image/OpenTerminal.JPG)
2. Run the following command in the terminal `pip install tensorboard boto3 tensorflow_io`
3. Run the notebook cell below to generate a command to launch TensorBoard
3. Copy the command and paste it into the terminal and hit Enter
4. Return to the notebook an click the link provided in the bellow cell

In [ ]:
from IPython.display import HTML
cur_dir = os.getcwd().replace(os.environ["HOME"],"")
HTML(f'''1. Paste the following command into the Studio Terminal <code style="background-color:gray;">tensorboard --logdir {tensorboard_output_config.s3_output_path}</code><br>
2. Click <a href='/jupyter/default/proxy/6006/'>here</a> to open TensorBoard''')

## Model Deployment
In this section we'll deploy our model to a SageMaker Endpoint. We'll then use the endpoint to generate summaries for random examples from the test set

In [ ]:
# We have to wait for the job to finish before we can deploy the model 
estimator.latest_training_job.wait(logs="None")

Once the training job has completed, we can deploy the model to a SageMaker Endpoint.
We will use a [Deep learning container for large model inference](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-dlc.html) for deployment which is optimized for serving large models in excess of 100B parameters

In [ ]:
# We'll need a few additional imports for model deployment
from sagemaker.model import Model
from sagemaker import serializers, deserializers

In [ ]:
# This is the docker image that will be used for inference
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.0-cu117"
)
print(f"Image going to be used is ---- > {inference_image_uri}")

The next step is to create a model deploymnent packages which will be used to deploy our model to a SageMaker Endpoint. The model deployment package is a tarball that contains the model artifacts, [inference code](src/inference/model.py), and any [additional dependencies](src/inference/requirements.txt) required to run the inference code. We'll go through the following steps to create the model deployment package:
1. Download the trained model artifact from S3 to the local filesystem
2. Cretae a `serving.properties` file that will configure our hosting environment
3. Combine the trained model, the inference code, and the `serving.properties` file into a tarball with the following structure:
```
|-- model.py         # inference code
|-- requirements.txt    # additional dependencies
|-- serving.properties  # configuration file
|-- <model_id>\         # model artifacts
    |-- config.json
    |-- pytorch_model.bin
    |-- special_tokens_map.json
    |-- tokenizer_config.json
    |-- tokenizer.json
    |-- vocab.json
```

In [ ]:
!aws s3 cp {estimator.model_data} .

Note that the model was trained using Low Rank Adaptation (LoRA), and as a result the model artifact is small (~10Mb) allowing us to repackage it along with our inference code. At deployment time, the base model will be downloaded from Hugging Face Hub and the LoRA weights will be applied to the base model. For deployment of larger models with LoRA weights, it is recommended to store the based model weights in your own S3 bucket.

In [ ]:
# extract the model artifacts into the inference code directory 
with tarfile.open("model.tar.gz", "r:gz") as tar:
    contents = tar.getnames()
    model_id = os.path.dirname(contents[-1]) # model id is the name of the folder containing the model files as generated by the training job
    tar.extractall("src/inference/")

In [ ]:
# generate the serving.properties file
# We'll use the python engine for inference and specify the model_id for the base model we want to use
with open("src/inference/serving.properties", "w") as f:
    f.write(
f"""engine=Python
option.model_id={model_id}
    """
    )

Now we have everything needed to create the model package. We'll combine the contents of the `src/inference` directory with the model artifact and create a tarball. We'll then upload the tarball to S3 and use the S3 URI to deploy the model.

In [ ]:
%%bash
cd src/
tar czvf model.tar.gz inference/
mv model.tar.gz ../

In [ ]:
hf_s3_code_artifact = sess.upload_data("model.tar.gz", bucket, f"{s3_key_prefix}/model")
print(f"S3 Code or Model tar ball uploaded to --- > {hf_s3_code_artifact}")

In [ ]:
def deploy_model(image_uri, model_data, role, endpoint_name, instance_type, sagemaker_session):
    """Helper function to create the SageMaker Endpoint resources and return a predictor"""
    model = Model(image_uri=image_uri, model_data=model_data, role=role)

    model.deploy(initial_instance_count=1, instance_type=instance_type, endpoint_name=endpoint_name)

    # our requests and responses will be in json format so we specify the serializer and the deserializer
    predictor = sagemaker.Predictor(
        endpoint_name=endpoint_name,
        sagemaker_session=sagemaker_session,
        serializer=serializers.JSONSerializer(),        # will convert python dict to json
        deserializer=deserializers.JSONDeserializer(),  # will convert json to python dict
    )

    return predictor

In [ ]:
# create a unique endpoint name
hf_endpoint_name = sagemaker.utils.name_from_base("t5-summarization")
print(f"Our endpoint will be called {hf_endpoint_name}")

In [ ]:
# deployment will take 5 to 10 minutes
hf_predictor = deploy_model(
    image_uri=inference_image_uri,
    model_data=hf_s3_code_artifact,
    role=role,
    endpoint_name=hf_endpoint_name,
    instance_type="ml.g4dn.xlarge",
    sagemaker_session=sess,
)

With the endpoint deployed, we can generate summaries on dialogues from the test dataset. We'll randomly select an examples and generate summaries. You can also provide your own dialogue to generate summaries just be sure to use the same format as the examples in the train dataset 

In [ ]:
from random import randint

In [ ]:
random_dialogue_idx = randint(0, test_data.shape[0])
random_dialogue = test_data["dialogue"][random_dialogue_idx]

output = hf_predictor.predict({"inputs": [random_dialogue], "parameters":{"max_length": 100}})
output_summary = output["outputs"][0]["summary_text"]

print("#####DIALOGUE######\n", random_dialogue)
print("\n#####GENERATED SUMMARY######\n", output_summary)

In [ ]:
# delete the endpoint when finished experimenting
hf_predictor.delete_endpoint()